### Killing process

In [14]:
!lsof -i :5000

In [13]:
!kill -9 3579


on WIN:

netstat -ano | findstr :5000

taskkill /PID <PID> /F

### Code

In [1]:
!pip freeze | grep mlflow

mlflow==2.17.0
mlflow-skinny==2.17.0


In [2]:
1+1

2

In [1]:
'''
Taking data from the database and saving it to a csv file
'''


import os
import psycopg
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"), 
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [2]:
df.head()

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,5851,6927-WTFIV,2018-04-01,2019-12-01,Month-to-month,No,Credit card (automatic),71.3,1389.20,Fiber optic,...,No,No,No,No,Male,1,No,No,No,1
1,5852,4118-CEVPF,2017-03-01,NaT,One year,Yes,Bank transfer (automatic),110.8,3836.30,Fiber optic,...,Yes,Yes,Yes,Yes,Female,1,No,No,Yes,0
2,5853,3398-ZOUAA,2018-01-01,2019-10-01,Month-to-month,Yes,Electronic check,69.1,1474.75,Fiber optic,...,No,No,No,No,Male,1,Yes,No,No,1
3,5854,9114-VEPUF,2014-12-01,NaT,One year,No,Electronic check,96.1,6001.45,Fiber optic,...,No,No,No,Yes,Male,0,Yes,No,Yes,0
4,5855,7876-BEUTG,2018-11-01,NaT,Month-to-month,Yes,Mailed check,48.8,720.10,DSL,...,No,No,No,No,Female,0,No,No,Yes,0


In [3]:
# 1. Название колонок вашего датафрейма запишите в текстовый файл
with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(','.join(df.columns.tolist()))

In [4]:
# 2. Подсчет статистик

counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}

for col in counts_columns:
		# посчитайте уникальные значения для колонок, где немного уникальных значений (переменная counts_columns)
    column_stat = df[col].value_counts().to_dict()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}

    # обновите словарь stats
    stats.update(column_stat)


stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max() # посчитайте максимальное значение в колонке
stats["monthly_charges_mean"] = df["monthly_charges"].mean() # посчитайте среднее значение в колонке
stats["monthly_charges_median"] = df["monthly_charges"].median() # посчитайте медианное значение в колонке
stats["total_charges_min"] = df["total_charges"].min() # посчитайте минимальное значение в колонке
stats["total_charges_max"] = df["total_charges"].max() # посчитайте максимальное значение в колонке
stats["total_charges_mean"] = df["total_charges"].mean() # посчитайте среднее значение в колонке
stats["total_charges_median"] = df["total_charges"].median() # посчитайте медианное значение в колонке
stats["unique_customers_number"] = df["customer_id"].nunique() # посчитайте кол-во уникальных id
stats["end_date_nan"] = df["end_date"].isna().sum() # посчитайте кол-во пустых строк в колонке

In [5]:

df.to_csv("users_churn.csv", index=False)

In [6]:
import mlflow

In [7]:
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [ ]:
!export $(cat .env | xargs)

In [13]:
# задаём название эксперимента и имя запуска для логирования в MLflow

EXPERIMENT_NAME = "churn_volkovandrey2"
RUN_NAME = "data_check2"

# создаём новый эксперимент в MLflow с указанным названием 
# если эксперимент с таким именем уже существует, 
# MLflow возвращает идентификатор существующего эксперимента
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id
    
    # логируем метрики эксперимента
    # предполагается, что переменная stats содержит словарь с метриками,
    # где ключи — это названия метрик, а значения — числовые значения метрик
    mlflow.log_metrics(stats)
    
    print(run_id)
    print(experiment_id)
    print(mlflow.get_artifact_uri())
    # логируем файлы как артефакты эксперимента — 'columns.txt' и 'users_churn.csv'
    mlflow.log_artifact('columns.txt', 'dataframe')
    mlflow.log_artifact('users_churn.csv', 'dataframe')


experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id)



# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert run.info.status == 'FINISHED'

# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов
os.remove('columns.txt')
os.remove('users_churn.csv')

In [10]:
mlflow.get_artifact_uri() 

_________________

In [12]:
mlflow.end_run()

In [2]:
import mlflow
experiment_id = '0'

with mlflow.start_run(run_name='log_model', experiment_id=experiment_id) as run:
    run_id = run.info.run_id
  
    model_info = mlflow.catboost.log_model(cb_model=model, artifact_path="models")

NameError: name 'model' is not defined

In [3]:
import mlflow
import numpy as np

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# напишите код, который подключает tracking и registry uri
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") # tracking uri
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") # registry uri


experiment_id = '0'

# указываем путь до окружения
pip_requirements="../requirements.txt"

# формируем сигнатуру, дополнительно передавая параметры применения модели
signature = mlflow.models.infer_signature(
	np.array([[0.1, 0.2, 0.3], [0.1, 0.2, 0.3]]),
	np.array([0.1, 0.2])
)
# формируем пример входных данных
input_example = [[0.1, 0.2, 0.3], [0.1, 0.2, 0.3]]
# предположим, мы хотим указать на то, что модель предсказывает на месяц вперёд
metadata = {"target_name": "churn"}
# путь до скрипта или ноутбука, который осуществляет обучение модели и валидацию
code_paths = ["train.py", "val_model.py"]


with mlflow.start_run(run_name="model_reg", experiment_id=experiment_id) as run:
    run_id = run.info.run_id
  
    model_info = mlflow.catboost.log_model(
        cb_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME,
        signature=signature,
        input_example=input_example,
        metadata=metadata,
        pip_requirements=pip_requirements,
        code_path=code_path
		)

2024/10/21 19:52:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run model_reg at: http://127.0.0.1:5000/#/experiments/0/runs/bfa2491e3d1a409c84d3205e366cad3b.
2024/10/21 19:52:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


NameError: name 'model' is not defined